In [1]:
import requests
import pandas as pd
import numpy as np
import string

In [2]:
data = str(requests.get('https://raw.githubusercontent.com/fspot/INFMDI-721/master/lesson5/products.csv').content.decode())
data = [line.split(";") for line in data.split("\n")]

In [3]:
data = pd.DataFrame(data[1:-1], columns=data[0])

In [4]:
data.head()

,username,ip_address,product,price,infos
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,May contain sugar
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,Contains peanut and fish
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish"


In [5]:
data.describe()

,username,ip_address,product,price,infos
count,200,200,200,200,200
unique,200,200,188,200,133
top,gbayles1v,204.196.159.175,"Pasta - Shells, Medium, Dry",342.82,Ingredients: sugar
freq,1,1,2,1,7


### Cleaning

In [6]:
def remove_punctuation(sentence):
    return ''.join(filter(lambda word: word not in string.punctuation, sentence))

def remove_stopwords(sentence, stop_words):
    return [word for word in sentence.split(" ") if word not in stop_words]

def convert(row, currencies):
    row['price'] = row['price'] / currencies[row['currency'].lower()]['rate']
    return row

def ip_to_country(ip):
    res = requests.get('https://freegeoip.app/json/{}'.format(ip))
    return res.json()['country_code'] if res else 'Undefined'

def country_to_currency(country, mapping):
    return mapping[country] if mapping.get(country) else 'Undefined'

def get_clean_df(df):
    # All APIs used to clean the dataframe
    mapping_country_to_currency = requests.get('http://country.io/currency.json').json()
    currencies = requests.get("http://www.floatrates.com/daily/eur.json").json()
    
    
    # Clean price column
    df['currency'] = 0
    df.loc[:, ('price', 'currency')] = df['price'].str.split(" ", expand=True).values
    df['price'] = pd.to_numeric(df['price'])

    # Get country informations
    df.loc[df['ip_address'] == 'nope', 'ip_address'] = np.NaN
    df['country'] = df.ip_address.map(ip_to_country)
    df.loc[df['currency'].isna(), 'currency'] = df.loc[df['currency'].isna(), 'country']\
                                                  .map(lambda country: country_to_currency(country, mapping_country_to_currency))
    
    # Apply conversion on the pice column
    currencies.update({'eur': {'rate': 1}, 'undefined': {'rate': 1}, 'byr': {'rate': 2.27}})
    df = df.apply(func=lambda row: convert(row, currencies), axis=1)
    
    
    # Clean infos column
    stop_words = ('and', 'contain', 'contains', 'may', 'ingredients', 'ingredient')
    df['infos'] = df['infos'].str.lower()
    df['infos'] = df.infos.map(remove_punctuation)
    df['infos'] = df.infos.map(lambda sentence: remove_stopwords(sentence, stop_words))
    # We flaten all list contained in the infos column (du to the previous splitting), then we group by index.
    dummies_df = pd.get_dummies(df.infos.explode()).groupby(level=0).agg(sum)
    df = pd.concat([df, dummies_df], axis=1).drop('infos', axis=1)
    
    return df

In [7]:
clean_df = get_clean_df(data.copy())

In [8]:
clean_df.head()

,username,ip_address,product,price,currency,country,egg,fish,gluten,milk,mustard,peanut,soja,sugar
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.800000,Undefined,Undefined,0,0,0,0,0,0,0,1
1,kizakov1,NaN,Soup - Campbells Bean Medley,379.260000,Undefined,Undefined,0,1,0,0,0,1,0,0
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.960000,Undefined,,0,1,0,0,1,0,0,0
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",314.323547,USD,US,0,0,1,0,0,0,0,0
4,mbuckney4,58.90.204.239,Radish - Pickled,7.852813,JPY,JP,1,1,0,0,0,0,0,1
